Loading data functions

In [93]:
import numpy as np
import pandas as pd

import os

import time

Catalog with microlensing events from GSA 2016--2021

In [94]:
file_name = "GSA_2016_2021_catalog.csv"
gsa_events = pd.read_csv(file_name, header = 0)

In [183]:
gsa_events

,#Name,Ra_deg,Dec_deg,Class,Remarks
0,Gaia16aua,213.1949,-59.5612,PSPL,NaN
1,Gaia16aye,295.0047,30.1315,bin,BHTOM TOI
2,Gaia16bnn,277.6759,-37.0714,PSPL,NaN
3,Gaia17ahl,260.0530,-43.9005,PSPL?,NaN
4,Gaia17ajg,269.4585,-31.5551,PSPL,BLG507.21.15236 OGLE-2017-BLG-0034
...,...,...,...,...,...
440,Gaia21fil,332.3580,50.6260,PSPL,BHTOM TOI
441,Gaia21fio,173.5408,-65.4620,PSPL,BHTOM TOI
442,Gaia21fjx,303.8140,22.4500,PSPL,ZTF21acegeyw OMEGA TOI BHTOM TOI
443,Gaia21fka,300.2858,32.4295,Par?,ZTF18abrrrzz OMEGA TOI


Cross-match catalogs with names in other surveys -- needed for loading the data

In [116]:
fin = "xmatch_cat/xmatch_gsa_ogleews.csv"
ews_known = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_ogle_mroz_bulge.csv"
ogle_known_A = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_ogle_mroz_disc.csv"
ogle_known_B = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_ogle_mroz_disc_possible.csv"
ogle_known_C = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_moa.csv"
moa_known = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_kmtnet.csv"
kmtnet_known = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_asassn.csv"
asassn_known = pd.read_csv(fin, header=0)
fin = "xmatch_cat/xmatch_gsa_ztf_alerts.csv"
ztfal_known = pd.read_csv(fin, header=0)

In [150]:
def get_ASASSN(name, band, asassn_known):
    res = asassn_known[asassn_known["name"] == name].index
    asassn_name = asassn_known["col1"].values[res[0]]
    phot_name = asassn_known["phot_file"].values[res[0]]
    file_name = "data/asassn_data/%s_%s.dat" % (phot_name, band)
    if os.path.isfile(file_name):
        data = np.genfromtxt(file_name, dtype=float, usecols=(0, 1, 2), unpack=True)
        return data
    else:
        return 0
    
def get_Gaia(name):
    file_name = "data/GSA/%s_%s.dat" % (name, name)
    data = np.genfromtxt(file_name, dtype=float, usecols=(0, 1, 2), unpack=True)
    return data
    
def get_KMTNet(name, obs, kmtnet_known):
    res = kmtnet_known[kmtnet_known["name"] == name].index
    kmtn_name = kmtnet_known["name_2"].values[res[0]]
    file_name = "data/KMTNet/%s_%s_%s.dat" % (name, kmtn_name, obs)
    if os.path.isfile(file_name):
        data = np.genfromtxt(file_name, dtype=float, usecols=(0, 1, 2), unpack=True)
        return data
    else:
        return 0
    
def get_MOA(name, moa_known):
    res = moa_known[moa_known["name"] == name].index
    moa_name = moa_known["name_2"].values[res[0]]
    data = np.genfromtxt("data/MOA/%s_%s.dat" % (name, moa_name), dtype=float, usecols=(0, 1, 2), unpack=True)
    return data

def get_OGLE_EWS(name, ogle_ews):
    res = ogle_ews[ogle_ews["name"] == name].index
    ogle_name = ogle_ews["name_2"].values[res[0]]
    data = np.genfromtxt("data/OGLE_EWS/%s_%s.dat" % (name, ogle_name), dtype=float, 
                         usecols=(0, 1, 2), unpack=True)
    return data

def get_OGLE_Mroz(name, ogle_known):
    res = ogle_known[ogle_known["name"] == name].index
    ogle_name = ogle_known["col1"].values[res[0]]
    file_name = "data/ogle_data/" + ogle_name + ".dat"
    data = np.genfromtxt(file_name, dtype=float, usecols=(0, 1, 2), unpack=True)
    return data

def get_ZTF_alerts(name, band, ztf_known):
    res = ztf_known[ztf_known["#Name"] == name].index
    ztfal_name = ztf_known["ZTFAlertName"].values[res[0]]
    file_name = "data/ZTFAlerts/%s_ZTFAl_%s_ZTF_al_%s.dat" % (name, ztfal_name, band)
    if os.path.isfile(file_name):
        data = np.genfromtxt(file_name, unpack=True)
        return data
    else:
        return 0

def get_ZTF(name, band):
    file_name = "data/ZTF/%s_ZTF_%s.dat" % (name, band)
    if os.path.isfile(file_name):
        data = np.genfromtxt(file_name, unpack=True)
        return data
    else:
        return 0

In [160]:
def get_followup(name):
    file_name = "data/BHTOM/%s.csv" % (name)
    if os.path.isfile(file_name):
        data = pd.read_csv(file_name, header=0, delimiter=';')
        return data
    else:
        return 0

In [161]:
name = "Gaia17ajg"
ogle_data = get_OGLE_Mroz(name, ogle_known_A)

name = "Gaia18cml"
ogle_ews_data = get_OGLE_EWS(name, ews_known)

name = "Gaia20dup"
moa_data = get_MOA(name, moa_known)

name = "Gaia18cxr"
obs = ["KMTS", "KMTC", "KMTA"]
kmtn_data = get_KMTNet(name, "KMTS", kmtnet_known)

name = "Gaia21efs"
band = ["g", "V"]
asassn_data = get_ASASSN(name, "V", asassn_known)

name = "Gaia16aua"
gaia_data = get_Gaia(name)

name = "Gaia19enj"
bands = ["g", "r", "i"]
ztf_al_data = get_ZTF_alerts(name, bands[0], ztfal_known)

name = "Gaia19axg"
bands = ["g", "r", "i"]
ztf_data = get_ZTF(name, bands[1])

name = "Gaia16aye"
fup_dataframe = get_followup(name)

In [182]:
for band in fup_dataframe["Filter"].unique():
    times = fup_dataframe[fup_dataframe["Filter"] == band]["JD"]
    mags = fup_dataframe[fup_dataframe["Filter"] == band]["Magnitude"]
    errs = fup_dataframe[fup_dataframe["Filter"] == band]["Error"]
    data = np.vstack((times, mags, errs))

4196
[[2.45761025e+06 2.45761025e+06 2.45761025e+06 ... 2.45918422e+06
  2.45918422e+06 2.45926563e+06]
 [1.66354561e+01 1.65483341e+01 1.65995960e+01 ... 1.83150520e+01
  1.83150997e+01 1.78115082e+01]
 [6.52287280e-02 5.94904418e-02 5.63278181e-02 ... 1.68529830e-01
  1.68529828e-01 1.74568975e-01]]
11877
[[2.45761025e+06 2.45761025e+06 2.45761025e+06 ... 2.45934044e+06
  2.45934457e+06 2.45934542e+06]
 [1.49139242e+01 1.49069786e+01 1.48965940e+01 ... 1.59290876e+01
  1.63859062e+01 1.65562420e+01]
 [4.99058365e-02 4.69481919e-02 4.39556469e-02 ... 8.19065234e-02
  1.17559399e-01 1.10658955e-01]]
16526
[[2.45761025e+06 2.45761025e+06 2.45761025e+06 ... 2.45968348e+06
  2.45968748e+06 2.45968947e+06]
 [1.41081238e+01 1.41017523e+01 1.41171265e+01 ... 1.58501148e+01
  1.51786699e+01 1.51204309e+01]
 [9.22435370e-02 9.05940080e-02 8.43306843e-02 ... 1.13608376e-01
  6.30889292e-02 5.63434916e-02]]
9374
[[2.45761025e+06 2.45761025e+06 2.45761026e+06 ... 2.45968748e+06
  2.45968947e+06 2